In [ ]:
while (!rdr.EndOfStream)
{
  string linea = rdr.ReadLine();
  //  DatoClima datosDia; --> a partir de C#8 podemos declarla en el mismo momento de usar el "out"

  //  procesar linea
  //
  var resultado = TryParseDatoClima(linea, out DatoClima datosDia);
  if (resultado.ok)
  {
    //  agregar datosDia al array de valores validos
    //
    datos.Add(resultado.valor);
  }
  else
  {
    Console.WriteLine($"Marcar error y seguir...{linea}");
  }
}

//float sumaTotal = 0.0f;

//foreach (var item in datos)
//  sumaTotal += item.TempMinima;
//Console.WriteLine($"Promedio de minimas: {sumaTotal / datos.Count:N1}");

float[] d = new float[datos.Count];
int idx = 0;
foreach (var x in datos)
  d[idx++] = x.TempMinima;

//  en d[] voy a tener el array de todos los valores de ....
//    temp minima

Calculo(Promedio, d, "Promedio de temperaturas minimas");
Calculo(Minimo, d, "Minimo de temp minimas");
Calculo(Maximo, d, "Maximo de temp minimas");


Console.ReadLine();

//  rdr.Close();    ==> OK, pero no estoy liberando recursos del OS, mejor Dispose()
//  rdr.Dispose();  ==> es automatico si uso la sentencia using

//  calcular estadisticas...

void Calculo(Func<float[], float> delegado, float[] valores, string texto)
{
  float resultado = delegado(valores);

  Console.WriteLine($"{texto}: {resultado}");
}


float Promedio(float[] items)
{
  float total = 0f;

  foreach (var i in items)
    total += i;
  return total / items.Length;
}

float Maximo(float[] items)
{
  float maximo = float.MinValue;

  foreach (var i in items)
    if (i > maximo)
      maximo = i;
  return maximo;
}

float Minimo(float[] items)
{
  float minimo = float.MaxValue;

  foreach (var i in items)
    if (i < minimo)
      minimo = i;
  return minimo;
}



//   name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
//   "Rosario, Argentina",2022-06-01,11.7,6.3,8.5,11.7,4.9,8.3,3.4,70.6,0,0,0,,0,0,16.2,8.5,131.3,1021.9,100,11.7,120.6,10.5,5,10,2022-06-01T07:57:09,2022-06-01T18:03:47,0.02,Overcast,Cloudy skies throughout the day.,cloudy,"SAAR,87480099999"
//                      ^                                                                                                                                                                                                      ^                   
//                      primera                                                                                                                                                                                                ultima
//
(bool ok, DatoClima valor) TryParseDatoClima(string linea, out DatoClima datosDia)
{
  datosDia = default;

  int primera = linea.IndexOf("\",");
  int ultima = linea.IndexOf(",\"");

  if (primera < 0 || ultima < 0 || ultima < primera)
    return (false, default);

  string lineaValida = linea.Substring(primera + 2, ultima - (primera + 2));
  string[] items = lineaValida.Split(new char[] { ',' }, StringSplitOptions.TrimEntries);

  if (items.Length != NUMERO_CAMPOS)
    return (false, default);

  if (!DateTime.TryParseExact(items[0], new[] { "yyyy-MM-dd" }, null, DateTimeStyles.None,
        out DateTime fecha))
  {
    Console.WriteLine($"Fecha incorrecta ==> {items[0]}");
    return (false, default);
  }

  //  para generar un error y probar try..catch, editar el .csv y cambiar alguno de los
  //  valores de temperatura para que no sea un float (ej 16xx.8)
  //
  try
  {
    float tempMaxima = ConvertirFloat(items[1], "Temperatura Maxima");
    float tempMinima = ConvertirFloat(items[2], "Temperatura Minima");
    float humedad = ConvertirFloat(items[8], "Humedad");

    //  datosDia.Fecha = fecha; NO!!!
    //
    datosDia = new DatoClima(fecha, tempMinima, tempMaxima, humedad);
    return (true, datosDia);
  }
  catch (LecturaClimaException ex)
  {
    Console.WriteLine(ex.Message);
    return (false, default);
  }

  //  funcion local --> puede lanzar LecturaClimaException
  //
  float ConvertirFloat(string s, string nombreCampo)
  {
    if (!float.TryParse(s, NumberStyles.Any, CultureInfo.InvariantCulture, out float resultado))
      throw new LecturaClimaException(nombreCampo, s);

    return resultado;
  }
}

/// <summary>
/// Tipo inmutable que contiene los datos climaticos para una fecha particular
/// </summary>
/// <param name="Fecha"></param>
/// <param name="TempMinima"></param>
/// <param name="TempMaxima"></param>
/// <param name="Humedad"></param>
public record DatoClima(DateTime Fecha, float TempMinima, float TempMaxima, float Humedad);

/// <summary>
/// Clase de excepcion customizada para nuestra aplicacion
/// Contiene el mensaje producido desde la conversion a DatoClima
/// </summary>
class LecturaClimaException : Exception
{
  public LecturaClimaException(string mensaje, string origen) : 
    base($"Error leyendo {mensaje} ==> valor original [{origen}]")
  {

  }
}



  //  var (ok, valor) = TryParseDatoClima(linea);
  if (TryParseDatoClima(linea) is ( true, var datoClima))
  {
    //  agregar datosDia al array de valores validos
    //
    datos.Add(datoClima);
  }
  else
  {
    Console.WriteLine($"Marcar error y seguir...{linea}");
  }



  if (TryParseDatoClima(linea, out DatoClima datosDia) is { ok: true } dato)
  {
    //  agregar datosDia al array de valores validos
    //
    datos.Add(dato.valor);
  }
  else
  {
    Console.WriteLine($"Marcar error y seguir...{linea}");
  }
